### Importing Necessary libraries

In [6]:
import pandas as pd
import numpy as np
import gensim
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pickle
import re 
import os
import spacy
import time
from webdriver_manager.chrome import ChromeDriverManager

### Scraping the data from NIKE website

In [2]:


# Empty lists to store the extracted data
description = []
link = []

# Dictionary containing the category names as keys and their respective URLs as values
url_dict = {
    'mens-tops-t-shirts': 'https://www.nike.com/in/w/mens-tops-t-shirts-9om13znik1',
    'womens-tops-t-shirts': 'https://www.nike.com/in/w/womens-tops-t-shirts-5e1x6z9om13',
    'hoodies-sweatshirts': 'https://www.nike.com/in/w/hoodies-sweatshirts-5e1x6z6riveznik1',
    'jackets-gilets': 'https://www.nike.com/in/w/jackets-gilets-50r7yz5e1x6znik1',
    'shorts': 'https://www.nike.com/in/w/shorts-38fphz5e1x6znik1',
    'trousers-tights': 'https://www.nike.com/in/w/trousers-tights-2kq19z5e1x6znik1',
    'womens-skirts-dresses': 'https://www.nike.com/in/w/womens-skirts-dresses-5e1x6z8y3qp',
    'womens-modest-wear': 'https://www.nike.com/in/w/womens-modest-wear-22fj5z5e1x6',
    'womens-back-to-basics': 'https://www.nike.com/in/w/womens-back-to-basics-5e1x6zkkjj'
}

# Get the path to the current directory
current_dir = os.path.dirname(os.path.abspath(__file__))

# Construct the path to the Chrome driver
driver_path = os.path.join(current_dir, "chromedriver.exe")

# Check if the driver file exists
if not os.path.isfile(driver_path):
    # If the driver file doesn't exist, download and install it using ChromeDriverManager
    driver_path = ChromeDriverManager().install()
 

# Web scraping for infinite scrolling page
for category, url in url_dict.items():
    driver = webdriver.Chrome(executable_path=driver_path) # Use the Chrome driver with the specified path
    driver.get(url)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 1  # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")  # Get the screen height of the web page
    i = 1

    while True:
        # Scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
        i += 1
        time.sleep(scroll_pause_time)
        # Update scroll height each time after scrolling, as the scroll height can change
        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 

    # Extract data from the page
    soup = BeautifulSoup(driver.page_source, "html.parser")
    description += [i.text for i in soup.find_all('div', class_="product-card__subtitle")]
    link += [a_tag['href'] for a_tag in soup.find_all('a', class_='product-card__img-link-overlay')]
    
    # Close the browser driver
    driver.quit()

# Create a DataFrame from the extracted data
data = {"description": description, "product_link": link}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('nike.csv', index=False)


### Scraping the data from LEVI's website

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# URL dictionary containing different sections of Levi's men's and women's clothing
url_dict1 = {
    'men-t-shirts-polos': 'https://www.levi.in/men/clothing/t-shirts-polos?prefn1=items_type&prefv1=Polos%7CT-Shirts&sz=482',
    'men-pants': 'https://www.levi.in/men/clothing/pants?sz=118',
    'men-shorts': 'https://www.levi.in/men/clothing/shorts?sz=57',
    'men-jackets': 'https://www.levi.in/men/clothing/jackets?sz=72',
    'men-shirts': 'https://www.levi.in/men/clothing/shirts?sz=504',
    'men-jeans': 'https://www.levi.in/men/clothing/jeans?sz=612',
    'women-jeans': 'https://www.levi.in/women/clothing/jeans?sz=569',
    'women-t-shirts-tops': 'https://www.levi.in/women/clothing/t-shirts-tops?sz=255',
    'women-shirts': 'https://www.levi.in/women/clothing/shirts?sz=32',
    'women-jackets': 'https://www.levi.in/women/clothing/jackets?sz=59',
    'women-shorts-skirts': 'https://www.levi.in/women/clothing/shorts-skirts?sz=26',
    'women-dresses-jumpsuits': 'https://www.levi.in/women/clothing/dresses-jumpsuits?sz=31',
    'women-pants': 'https://www.levi.in/women/clothing/pants?sz=36'
}

# Initialize empty lists to store the data
title = []
description = []
link = []

# Iterate over each URL in the URL dictionary
for section, url in url_dict1.items():
    # Send a GET request to the website
    response = requests.get(url)
    
    # Print the response status code
    print(response)
    
    # Create a BeautifulSoup object to parse the HTML content
    soup = bs(response.text)
    
    # Extract the description and link of each product using appropriate HTML tags and attributes
    description += [i.text.strip('\n') for i in soup.find_all('a', attrs={'class': 'name-link'})]
    homepage = 'https://www.levi.in'
    link += [homepage + a_tag['href'] for a_tag in soup.find_all('a', attrs={'class': 'name-link'})]

# Create a dictionary containing the extracted data
data1 = {"description": description, "product_link": link}

# Create a pandas DataFrame using the dictionary
df_levis = pd.DataFrame(data1)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


(2789, 2)

#### Merging two dataframe 

In [ ]:
# Read the 'nike.csv' file into a pandas DataFrame
df_nike = pd.read_csv('nike.csv')

# Get the shape (number of rows and columns) of the DataFrame
shape = df_nike.shape


(1015, 2)

In [ ]:

# Append the 'df_levis' DataFrame to 'df_nike' DataFrame
combined_df = df_nike.append(df_levis)

# Write the combined DataFrame to a CSV file named 'clothing_similarity.csv' without including the index
combined_df.to_csv('clothing_similarity.csv', index=False)


In [8]:
# Read the 'clothing_similarity.csv' file from the specified path into a new DataFrame named 'df'
df = pd.read_csv('/content/drive/MyDrive/clothing_similarity.csv')

# Randomly shuffle the rows of the DataFrame
df = df.sample(frac=1)

# Display the first few rows of the shuffled DataFrame
df.head()

,description,product_link
3529,Women's Regular T-Shirt,https://www.levi.in/women/clothing/t-shirts-to...
1990,Men's Geometric Print Slim Fit Shirt,https://www.levi.in/men/clothing/shirts/mens-g...
3411,721 Redloop™ High Rise Skinny Jeans,https://www.levi.in/women/clothing/jeans/721-r...
3048,Women's Mid Rise 711 Skinny Fit Jeans,https://www.levi.in/women/clothing/jeans/women...
1584,Men's Beige Slim Tapered Fit Trousers,https://www.levi.in/men/clothing/pants/mens-be...


### Preprocessing the data

In [9]:

# Define a function to remove "Levi's®" from a string and strip extra whitespace
def preprocess(string):
    regex = re.compile("Levi's®")
    return regex.sub('', string).strip(' ')

# Apply the 'preprocess' function to the 'description' column of the DataFrame
df['description'] = df['description'].apply(preprocess)

# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

# Define a function to preprocess text using spaCy
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()

    # Tokenize the text using spaCy
    doc = nlp(text)

    # Remove stopwords and lemmatize tokens
    tokens = [token.lemma_ for token in doc if not token.is_stop]

    return tokens


## Building the model

In [ ]:

# Preprocess the text corpus
text_corpus = df['description']
processed_corpus = [preprocess_text(doc) for doc in text_corpus]

# Create a dictionary of unique words in the corpus
dictionary = gensim.corpora.Dictionary(processed_corpus)

# Convert the corpus into Bag-of-Words representation
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

# Compute TF-IDF weights for the corpus
tfidf = gensim.models.TfidfModel(bow_corpus, smartirs='npu')
tfidf_corpus = tfidf[bow_corpus]

# Build an index for similarity calculations
index = gensim.similarities.MatrixSimilarity(tfidf_corpus)

# Save the objects (dictionary, tfidf, index) into a single pickle file
with open('clothing_similarity.pkl', 'wb') as f:
    pickle.dump((dictionary, tfidf, index), f)


In [ ]:

# Define the ranked_results function that accepts a text query and returns the ranked suggestions
def ranked_results(text, n):
    # Load the objects (dictionary, tfidf, index) from the pickle file
    with open('clothing_similarity.pkl', 'rb') as f:
        dictionary, tfidf, index = pickle.load(f)

    # Preprocess the new document/query
    new_doc = preprocess_text(text)

    # Convert the preprocessed document to Bag-of-Words representation
    new_vec = dictionary.doc2bow(new_doc)

    # Compute TF-IDF weights for the new document
    vec_bow_tfidf = tfidf[new_vec]

    # Calculate similarities between the new document and the corpus
    sims = index[vec_bow_tfidf]

    # Sort the similarities in descending order and select the top 'n' results
    # Generate a generator expression to pair the index and similarity values
    sorted_sims = ((idx, similarity) for idx, similarity in enumerate(sims))

    # Sort the similarities in descending order based on the similarity value
    # The 'key' parameter specifies that we want to sort based on the second item in each pair (similarity)
    # The lambda function -item[1] negates the similarity value to achieve descending order
    # Select the top 'n' results using slicing
    sorted_sims = sorted(sorted_sims, key=lambda item: -item[1])[:n]


    # Print the ranked results with their similarity scores
    for idx, similarity in sorted_sims:
        product_link = df['product_link'].iloc[idx]
        print({product_link: round(similarity, 2)})

# Calling the ranked_results function with a sample text query
# Test Case 1
ranked_results("Crew-neck sweatshirt for men.",5)

{'https://www.nike.com/in/t/jordan-brooklyn-fleece-crew-neck-sweatshirt-Ctt2nH/DQ7520-485': 0.84}
{'https://www.nike.com/in/t/jordan-brooklyn-fleece-crew-neck-sweatshirt-Ctt2nH/DQ7520-485': 0.84}
{'https://www.nike.com/in/t/jordan-two-18-crew-neck-sweatshirt-LVDJzV/DV6966-687': 0.83}
{'https://www.nike.com/in/t/jordan-shelflife-crew-neck-sweatshirt-mFtjbW/DV7014-206': 0.83}
{'https://www.nike.com/in/t/jordan-titan-crew-neck-sweatshirt-qFtdj8/DV7040-819': 0.83}


In [ ]:
# Test Case 2
ranked_results("joggers for women",6)

{'https://www.levi.in/women/clothing/pants/levis-women-joggers/A11950003.html': 1.0}
{'https://www.levi.in/women/clothing/pants/levis-women-joggers/A11950002.html': 1.0}
{'https://www.nike.com/in/t/sportswear-tech-fleece-joggers-h1ntxv/CU4496-491': 0.99}
{'https://www.levi.in/men/clothing/pants/levis-mens-joggers/A31870002.html': 0.99}
{'https://www.levi.in/men/clothing/pants/levis-mens-jogger-fit-joggers/A07250005.html': 0.98}
{'https://www.levi.in/women/clothing/jeans/levis-womens-slim-fit-joggers/A13420004.html': 0.92}


In [ ]:
# Test Case 3
ranked_results("Women's Running Leggings",5)

{'https://www.nike.com/in/t/epic-fast-mid-rise-running-leggings-L0MXcm/CZ9241-655': 0.83}
{'https://www.nike.com/in/t/epic-fast-mid-rise-running-leggings-L0MXcm/CZ9241-655': 0.83}
{'https://www.nike.com/in/t/epic-fast-mid-rise-running-leggings-L0MXcm/CZ9241-010': 0.83}
{'https://www.nike.com/in/t/dri-fit-swoosh-run-running-shorts-65klmb/DQ6361-430': 0.62}
{'https://www.nike.com/in/t/tempo-running-shorts-JxDWvp/CU8893-272': 0.62}


In [ ]:
# Test Case 4
ranked_results("men's Hoodie",10)

{'https://www.nike.com/in/t/forward-hoodie-hoodie-qKbJFt/DX4303-084': 1.0}
{'https://www.nike.com/in/t/forward-hoodie-hoodie-qKbJFt/DX4303-077': 1.0}
{'https://www.nike.com/in/t/jordan-artist-series-by-parker-duncan-hoodie-hwPhKF/DX0387-274': 0.99}
{'https://www.nike.com/in/t/forward-hoodie-hoodie-0hGfRb/DR4529-077': 0.99}
{'https://www.nike.com/in/t/jordan-artist-series-by-parker-duncan-hoodie-hwPhKF/DX0387-274': 0.99}
{'https://www.nike.com/in/t/forward-hoodie-hoodie-0hGfRb/DR4529-077': 0.99}
{'https://www.nike.com/in/t/forward-hoodie-hoodie-0hGfRb/DR4529-077': 0.99}
{'https://www.nike.com/in/t/jordan-artist-series-by-parker-duncan-hoodie-hwPhKF/DX0387-274': 0.99}
{'https://www.levi.in/men/clothing/jackets/levis-mens-hoodie-jacket/A45440000.html': 0.79}
{'https://www.levi.in/men/clothing/jackets/levis-mens-hoodie-jacket/A45440001.html': 0.79}


In [ ]:
# Test Case 5
ranked_results("womens blue skirts",5)

{'https://www.nike.com/in/t/naomi-osaka-skirt-q16CNk/DX1843-113': 0.95}
{'https://www.nike.com/in/t/dri-fit-advantage-tennis-skirt-FZwNLZ/DX1133-010': 0.74}
{'https://www.nike.com/in/t/nikecourt-dri-fit-slam-tennis-skirt-fp1tvQ/DR9750-655': 0.74}
{'https://www.levi.in/women/clothing/shorts-skirts/levis-womens-pride-skirt/A26540000.html': 0.63}
{'https://www.nike.com/in/t/nikecourt-dri-fit-victory-flouncy-tennis-skirt-V0cwGg/DH9553-100': 0.53}
